## Haskell Test Framework:

The `Haskell Test Framework` (`HTF`) supplies unit tests via `HUnit` and<br>
property tests via `QuickCheck`. The following code describes some of the<br>
functionality one may find in the test suite for the `Attenuations` project.<br>
some other useful tutorials can be found here:
* <a href="http://hackage.haskell.org/package/HTF-0.13.2.4/docs/Test-Framework-Tutorial.html">HTF hackage tutorial</a>
* <a href="https://wiki.haskell.org/HUnit_1.0_User%27s_Guide">HUnit user guide for assertion details</a>
* <a href="https://github.com/skogsbaer/HTF/tree/master/sample">A directory structure example</a>
* <a href="http://www.cse.chalmers.se/~rjmh/QuickCheck/manual.html">A QuickCheck guide</a>




At the top level one can safely disentangle functionality from testing<br>
by creating a `RayTracer` directory for the former and a `Tests` directory<br>
for the latter. Now by creating a `TestMain.hs` with the following lines,<br>
one can import the tests and the modules independently as-well-as define<br>
sub-suites to run individually.<p>
```
{-# OPTIONS_GHC -F -pgmF htfpp #-}

module AttenuationTest where
import RayTracer.RayLength
import RayTracer.Lattice
import RayTracer.Rhythm
import Test.Framework

import {-@ HTF_TESTS @-} Tests.IndexerTests
import {-@ HTF_TESTS @-} Tests.SymmetryTests
import {-@ HTF_TESTS @-} Tests.XRegionTests

main = htfMain htf_importedTests

symmetryTests = htfMain htf_Tests_SymmetryTests_thisModulesTests
indexerTests = htfMain htf_Tests_IndexerTests_thisModulesTests
xRegionTests = htfMain htf_Tests_XRegionTests_thisModulesTests
```

### Some Subtleties.
__QuickCheck versus HUnit:__<p>
From `ghci` one can perform a `quickCheck` on any property test, `prop_` prefixed,<br>
by running something akin to `quickCheck prop_someProperty`. To run a specific unit<br>
test appears a bit more tricky. One can either run the whole suite or another way<br>
I have seen is to import HUnit and define directly:
```
import Test.HUnit

test1 = TestCase test_rabbits
test2 = TestCase test_someotherFunction

tests = TestList [TestLabel "rabbits" test1, TestLabel "other function" test2]

runTestTT tests
```
Which then returns something like:
```
Cases: 2  Tried: 2  Errors: 0  Failures: 0
Counts {cases = 2, tried = 2, errors = 0, failures = 0}
```
<p>
__Limiting the Range of Test Data:__<p>

It may often be the case that a functions domain of validity is limited<br>
to a small subset of its range and testing outside of that range isn't<br>
very useful. The `choose` function makes it possible to limit the range<br>
of test values while maintaining statistical randomness. For instance,<br>
verifying that `cos (π/2-θ)` is the same as `sin θ` for values between<br>
0 and 2π can be tested:<p>
```
tol :: Double -> Integer
tol d = round $ d * 10^12

prop_cosToSin = do
  θ <- choose (0, 2*pi)
  return $ (tol.cos) (pi/2 - θ) == (tol.sin) θ
```
where `tol` allows for some wiggle room in the approximation.


### Playing and then Replaying a test.

In theory, one would state the following Pragma and import the following modules:<p>
```
{-# OPTIONS_GHC -F -pgmF htfpp #-}

module Tests.SymmetryTests where
import RayTracer.RayLength
import Test.Framework
```
Since Jupyter does not yet support Pragmas imaginations must be used.<br>
Then the following code can be `tested` and `retested` using the following<br>
constructions.<p>
```
prop_rot4Id x y = do
  t <- choose (0, 2*pi)
  let coords = ((x, y),t)
  let rotFour = foldr (.) id [rot90 | x<-[1..4]]
  return $ (mtol.divPi.rotFour) coords == mtol coords
  where
    divPi ((x, y),theta) = ((x, y), theta - (2*pi))
    mtol ((x,y), t) = ((ttol x, ttol y), ttol t)
    ttol d = round $ d * 10^12 -- not very good accuracy.

prop_rot4IdReplay =
  withQCArgs (\a -> a { replay = read replayArg })
  prop_rot4Id
  
replayArg = "Just (TFGenR 15067B55359906C0776B9C0A73ACEE7D9C124B4AE3DAC3AFCB451E04B1EF7BD1 0 31 5 0,28)"

main = htfMain htf_Tests_SymmetryTests_thisModulesTests
 ```



### Fairly General Algebraic Testing:

One of the coolest examples I have seen this far concerns testing<br>
algebraic properties like the associativity of function composition<br>
By importing `Text.Show.Functions` one can even test this property<br>
with amazingly clean style:

```
import Text.Show.Functions

prop_ComposeAssoc f g h x =
  ((f . g) . h) x == (f . (g . h)) x
  where types = [f, g, h] :: [Int->Int]
```